In [2]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN
from keras import layers

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [5]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [6]:
ctable = CharacterTable(chars)

In [7]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

In [11]:
# 讀檔
testData = np.load("dataset/trainData.npy")[1980000:]
testTarget = np.load("dataset/trainTarget.npy")[1980000:]
print(len(testData))

trainData = np.load("dataset/trainData.npy")[0:1000000]
trainTarget = np.load("dataset/trainTarget.npy")[0:1000000]

20000


In [12]:
from keras.layers import *

model = Sequential()
model.add(LSTM(256, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(4))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(13, activation='softmax'))  # 4x13 = 52
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_3 (Dense)              (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print("~~~")
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(trainData, trainTarget,
              epochs=1,
              batch_size=256,
              shuffle=True,
              validation_data=(testData, testTarget),
              verbose =2,
              )
    for i in range(10):
        ind = np.random.randint(0, len(testData))
        rowx, rowy = testData[np.array([ind])], testTarget[np.array([ind])]
        preds = model.predict(rowx)[0]
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

~~~

--------------------------------------------------
Iteration 0
Train on 200000 samples, validate on 20000 samples
Epoch 1/1
 - 1935s - loss: 1.3331 - acc: 0.5002 - val_loss: 2.9595 - val_acc: 0.2288
Q 993-273 T 720  ☒ 377 
Q 995+94  T 1089 ☒ 100 
Q 991-301 T 690  ☒ 470 
Q 994-420 T 574  ☒ 527 
Q 994+141 T 1135 ☒ 170 
Q 993-602 T 391  ☒ 674 
Q 996-268 T 728  ☒ 327 
Q 992-246 T 746  ☒ 377 
Q 990-49  T 941  ☒ 570 
Q 993-90  T 903  ☒ 970 

--------------------------------------------------
Iteration 1
Train on 200000 samples, validate on 20000 samples
Epoch 1/1
 - 1717s - loss: 0.9449 - acc: 0.6522 - val_loss: 3.8379 - val_acc: 0.2299
Q 999-275 T 724  ☒ 321 
Q 998-103 T 895  ☒ 208 
Q 991+988 T 1979 ☒ 1055
Q 991-72  T 919  ☒ 703 
Q 993+419 T 1412 ☒ 581 
Q 990+57  T 1047 ☒ 178 
Q 999-850 T 149  ☒ 181 
Q 990+304 T 1294 ☒ 481 
Q 991-127 T 864  ☒ 208 
Q 992-681 T 311  ☒ 781 

--------------------------------------------------
Iteration 2
Train on 200000 samples, validate on 20000 samples
E

Q 991+565 T 1556 ☒ 685 
Q 997+920 T 1917 ☒ -070
Q 995+52  T 1047 ☒ 658 
Q 990-881 T 109  ☒ -89 
Q 990+837 T 1827 ☒ 973 
Q 990+578 T 1568 ☒ 686 
Q 998-896 T 102  ☒ -84 
Q 995-510 T 485  ☒ -87 
Q 990+543 T 1533 ☒ 673 

--------------------------------------------------
Iteration 16
Train on 200000 samples, validate on 20000 samples
Epoch 1/1
 - 317s - loss: 0.0146 - acc: 0.9962 - val_loss: 10.2484 - val_acc: 0.1978
Q 992-282 T 710  ☒ 374 
Q 996-285 T 711  ☒ 343 
Q 997+229 T 1226 ☒ 331 
Q 993-22  T 971  ☒ 178 
Q 990+465 T 1455 ☒ 584 
Q 991-906 T 85   ☒ -90 
Q 997+729 T 1726 ☒ 761 
Q 997-142 T 855  ☒ -91 
Q 990-76  T 914  ☒ 795 
Q 997+523 T 1520 ☒ 671 

--------------------------------------------------
Iteration 17
Train on 200000 samples, validate on 20000 samples
Epoch 1/1
